In [100]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics

In [2]:
test_df = pd.read_csv("smsspam_test.csv",encoding='utf-8')
test_df.head()

,000,10,100,1000,10p,11,12hrs,150,150p,150ppm,...,xxx,ya,yeah,year,years,yes,yesterday,yo,yup,_label_
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train_df = pd.read_csv("smsspam_train.csv",encoding='utf-8')
train_df.head()

,000,10,100,1000,10p,11,12hrs,150,150p,150ppm,...,xxx,ya,yeah,year,years,yes,yesterday,yo,yup,_label_
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [197]:
#features_test = train_df.iloc[:,:500].values
features=train_df.drop('_label_', axis = 1)

In [198]:
features_test.shape

(3734, 500)

In [199]:
target = train_df.iloc[:,500].values
target.shape

(3734,)

In [50]:
classifier = BernoulliNB()

In [51]:
classifier.fit(features, target)

BernoulliNB()

In [55]:
features_test=test_df.drop('_label_', axis = 1)

In [56]:
target_test = test_df.iloc[:,500].values

In [57]:
unique, counts = np.unique(target_test, return_counts=True)
dict(zip(unique, counts))

{0: 1599, 1: 241}

In [58]:
y_pred = classifier.predict(features_test)

In [59]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [60]:
unique, counts = np.unique(y_pred, return_counts=True)
dict(zip(unique, counts))

{0: 1613, 1: 227}

In [61]:
with open('nb_predictions.txt', 'w', newline='') as file:
    for output in y_pred:
        file.write(str(output) + '\n')

In [62]:
model = LogisticRegression()

In [63]:
model.fit(features, target)

LogisticRegression()

In [64]:
lr_predict = model.predict(features_test)
lr_predict

array([0, 0, 0, ..., 0, 0, 0])

In [65]:
unique, counts = np.unique(lr_predict, return_counts=True)
dict(zip(unique, counts))

{0: 1625, 1: 215}

In [66]:
with open('lg_predictions.txt', 'w', newline='') as file:
    for output in lr_predict:
        file.write(str(output) + '\n')

In [123]:
def compute_score (test_score: np.array, predict_score: np.array)-> float:
    confusion_metrics = metrics.confusion_matrix(test_score,predict_score)
    tp = confusion_metrics[1][1]
    tn = confusion_metrics[0][0]
    fp = confusion_metrics[0][1]
    fn = confusion_metrics[1][0]
    
    precision = tp / (tp + fp)
    recall = tp /(tp + fn)
                
    f1_score = 2 * (precision* recall)/ (precision + recall)
    
    return f1_score

In [126]:
#f1_score(target_test,y_pred)

In [127]:
#f1_score(target_test,lr_predict)

In [128]:
compute_score(target_test,lr_predict)

0.9166666666666667

In [24]:
#accuracy = sum(lr_predict==target_test)/float(len(target_test))
#accuracy

In [25]:
#np.count_nonzero(lr_predict==target_test)

In [26]:
#len(target_test)

In [129]:
dt_predictions = np.loadtxt("dt_predictions.txt",dtype = 'int64')
dt_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [130]:
unique, counts = np.unique(dt_predictions, return_counts=True)
dict(zip(unique, counts))

{0: 1594, 1: 246}

In [131]:
#f1_score(target_test, dt_predictions)

In [132]:
compute_score(target_test,dt_predictions)

0.8870636550308009

In [171]:
combine = pd.DataFrame(y_pred, columns = ['Naive'])
combine.head(10)

,Naive
0,0
1,0
2,0
3,1
4,0
5,0
6,1
7,0
8,0
9,0


In [172]:
combine['LR_Predict'] = pd.DataFrame(y_pred)
combine.head(10)

,Naive,LR_Predict
0,0,0
1,0,0
2,0,0
3,1,1
4,0,0
5,0,0
6,1,1
7,0,0
8,0,0
9,0,0


In [173]:
combine['Decision'] = pd.DataFrame(dt_predictions)
combine.head(10)

,Naive,LR_Predict,Decision
0,0,0,0
1,0,0,0
2,0,0,0
3,1,1,0
4,0,0,0
5,0,0,0
6,1,1,1
7,0,0,0
8,0,0,0
9,0,0,0


In [174]:
np.count_nonzero(dt_predictions==target_test)

1785

In [175]:
combine['Final'] = combine['Naive'] + combine['LR_Predict'] + combine['Decision']

In [176]:
combine.head(10)

,Naive,LR_Predict,Decision,Final
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,1,1,0,2
4,0,0,0,0
5,0,0,0,0
6,1,1,1,3
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [177]:
combine.loc[combine['Final'] <= 1,'Final'] = 0

In [178]:
combine.loc[combine['Final'] >= 2,'Final'] = 1

In [168]:
#combine.loc[combine['Final'] == 3,'Final'] = 1

In [135]:
#combine.loc[combine['Final'] == 2,'Final'] = 1

In [163]:
#combine.loc[combine['Final'] == 3,'Final'] = 1
combine.head(10)

,Naive,LR_Predict,Decision,Final
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,1,1,0,1
4,0,0,0,0
5,0,0,0,0
6,1,1,1,1
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [179]:
voted_prediction = combine['Final'].values
voted_prediction.shape

(1840,)

In [180]:
confusion_metrics = metrics.confusion_matrix(target_test,voted_prediction)
confusion_metrics

array([[1590,    9],
       [  23,  218]])

In [181]:
compute_score(target_test, voted_prediction)

0.9316239316239316